# Notes #
1. Data preparation
    + Issues
        - Number of rows less than expected -- DONE
        - Dealing with missing values -- DONE
        - Converting target into binary 
        - Should I consider doing feature selection?
    + TODO
        - Add data exploratory steps
        - Show the data distribution etc
        - Add frequency of categorical fields
        - Add class imbalance details
2. Keras model
    - **Accuracy in the first epoch itself provides optimum results.  
    Check why this could be happening**
3. Scikit model
    - Note -- Check Keras wrapper for cross validation
4. Validation


**Rough notes**
1. scikit DONE
2. read
3. cuda
4. fix errors Later, since performance is close
5. Confusion matrix, ROC and validation

---

**Data preparation**

In [2]:
# Load dataset info #
import automation_script
import pandas as pd
import numpy as np
from os import path

dataset_name = "UCI Adult Salary"
dataset_info = automation_script.get_url(dataset_name)

In [3]:
# Data preparation #

names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
         'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
         'hours-per-week', 'native-country', 'target']
url = "../data/iris.csv" if path.exists("../data/dataset.csv.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=" ", header=None, names=names)

# Check for columns that contain missing values #
# col_names = data.columns
# print(col_names)
# num_data = data.shape[0]
# for c in col_names:
#     num_non = data[c].isin(["?"]).sum()
#     if num_non > 0:
#         print (c)
#         print (num_non)
#         print ("{0:.2f}%".format(float(num_non) / num_data * 100))
#         print ("\n")

data = data[data["workclass"] != "?"]
data = data[data["occupation"] != "?"]
data = data[data["native-country"] != "?"]

# Convert categorical fields #
categorical_col = ['workclass', 'education', 'marital-status', 'occupation',
                   'relationship', 'race', 'sex', 'native-country', 'target']
    
# for col in categorical_col:
#     categories = unique_of(data.col)
#     num_cat = count(categories)
#     for cat in categories:
#         data.col[cat] = index_of(cat in categories)

for col in categorical_col:
    b, c = np.unique(data[col], return_inverse=True)
    data[col] = c

feature_list = names[:14]
# Test train split #
X = data.loc[:, feature_list]
Y = data[['target']]

# Split the dataset into test and train datasets

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

# data.shape
# data.head()
type(Y)


pandas.core.frame.DataFrame

---

**Keras Model**

In [4]:
config = {
    'epoch': 10,
    'batch_size': 100,
    'verbose': 0,
    'model_info': {
        'loss':'binary_crossentropy',
        'optimizer':'adam',
        'metrics':['accuracy']
    }
}

keras_score,keras_params = automation_script.get_keras_params(X,Y,dataset_info,config)


Using TensorFlow backend.


27134/27134 [==============================] - 1s 25us/step

acc: 75.08%


---

**Scikit model**

In [5]:
scikit_score, scikit_params = automation_script.get_scikit_params(X,Y)

C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7865777253630132


---

**Kfold validation**

In [6]:
config = {
    'epoch': 10,
    'batch_size': 100,
    'splits':10,
    'model_info': {
        'loss':'binary_crossentropy',
        'optimizer':'adam',
        'metrics':['accuracy']
    }
}

kfold_acc = automation_script.get_kfold(X,Y,config)

acc: 75.22%
acc: 24.78%
acc: 24.25%
acc: 24.78%
acc: 24.79%
acc: 75.21%
acc: 24.79%
acc: 75.23%
acc: 24.77%
acc: 75.23%
44.91% (+/- 24.75%)

   -------------   



<Figure size 640x480 with 1 Axes>

[[20371     0]
 [ 6763     0]]

   -------------   

             precision    recall  f1-score   support

    Class 1       0.75      1.00      0.86     20371
    Class 2       0.00      0.00      0.00      6763

avg / total       0.56      0.75      0.64     27134



C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---

#### Write to Mastersheet ####

In [8]:
dataset_info['scikit_params'] = scikit_params
dataset_info['keras_params'] = keras_params
dataset_info['type'] = 'Binary'
accuracy_values = {
    'keras': keras_score,
    'scikit': scikit_score,
    'kfold': kfold_acc
}

automation_script.write_to_mastersheet(dataset_info,X,Y,accuracy_values)

---

# Pending #
1. See if you can fine tune keras model(75% acc) to match scikit model(78% acc)
2. Exploratory analysis of the dataset
3. Latex explanations
4. **Cross check what's wrong with ROC  
and Confusion matrix**